In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
df_fake = pd.read_csv("/content/Fake[1].csv")
df_true = pd.read_csv("/content/True[1].csv")

# New section

In [4]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
df_fake["class"] = 0
df_true["class"] = 1

In [11]:
df_fake.shape, df_true.shape

((23471, 5), (4532, 5))

In [12]:
df_fake_manual_testing = df_fake.tail(10)

# Drop rows only if the index exists
for i in range(23480, 23470, -1):
    if i in df_fake.index:
        df_fake.drop([i], axis=0, inplace=True)

# Fetch last 10 rows for true data manual testing
df_true_manual_testing = df_true.tail(10)

# Drop rows only if the index exists
for i in range(21416, 21406, -1):
    if i in df_true.index:
        df_true.drop([i], axis=0, inplace=True)

In [13]:
df_fake.shape, df_true.shape

((23471, 5), (4532, 5))

In [14]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-14-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-14-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [15]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",0
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",0
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",0
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",0
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",0
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",0
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",0
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",0
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",0
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",0


In [16]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
4522,Top Senate Democrat says unlikely Trump court ...,WASHINGTON (Reuters) - The top Democrat in the...,politicsNews,"April 2, 2017",1
4523,Democrat Senator Jon Tester will vote against ...,WASHINGTON (Reuters) - Democratic U.S. Senator...,politicsNews,"April 3, 2017",1
4524,Trump national security aides complete North K...,WASHINGTON (Reuters) - President Donald Trump’...,politicsNews,"April 2, 2017",1
4525,Flynn did not initially disclose income from R...,"WASHINGTON (Reuters) - Michael Flynn, Presiden...",politicsNews,"April 2, 2017",1
4526,"Schiff says Trump wants focus on tweets, not R...",WASHINGTON (Reuters) - The top Democrat on the...,politicsNews,"April 2, 2017",1
4527,Democrat Donnelly to support Trump pick for U....,WASHINGTON (Reuters) - Democratic U.S. Senator...,politicsNews,"April 2, 2017",1
4528,EPA chief says Paris climate agreement 'bad de...,WASHINGTON (Reuters) - The United States shoul...,politicsNews,"April 2, 2017",1
4529,Top Democrat on House panel says he has seen c...,WASHINGTON (Reuters) - The top Democrat on the...,politicsNews,"March 31, 2017",1
4530,Gritty New Jersey a source of wealth for Trump...,"WASHINGTON (Reuters) - Roadside apartments, in...",politicsNews,"April 2, 2017",1
4531,Wealthy White House staffers' finances detaile...,NaN,NaN,NaN,1


In [17]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [18]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [19]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [20]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [21]:
df.isnull().sum()

,0
text,1
class,0


In [22]:
df = df.sample(frac = 1)

In [23]:
df.head()

,text,class
15698,,0
13462,Trump talked about a country on the brink of d...,0
18300,Walmart was apologizing on Monday after an app...,0
13124,"While Trump works like crazy to win your vote,...",0
2887,WASHINGTON (Reuters) - U.S. Secretary of State...,1


In [24]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [25]:
df.columns

Index(['text', 'class'], dtype='object')

In [26]:
df.head()

,text,class
0,,0
1,Trump talked about a country on the brink of d...,0
2,Walmart was apologizing on Monday after an app...,0
3,"While Trump works like crazy to win your vote,...",0
4,WASHINGTON (Reuters) - U.S. Secretary of State...,1


In [29]:

def wordopt(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\[.*?\]', '', text)
        text = re.sub(r"\\W", " ", text)
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'<.*?>+', '', text)
        text = re.sub(r'[^a-zA-Z]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    else:
        return text

In [30]:
df["text"] = df["text"].apply(wordopt)

In [31]:

x = df["text"]
y = df["class"]

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [34]:
x_train = x_train.dropna()
x_test = x_test.dropna()

In [35]:
x_train = x_train.fillna('')
x_test = x_test.fillna('')

In [40]:
x_train = x_train.fillna('')
x_test = x_test.fillna('')

# Apply TF-IDF vectorization
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [42]:
print(xv_train.shape)  # Should return (21001, n_features) or similar
print(y_train.shape)

(21001, 80142)
(21002,)


In [43]:
y_train = y_train[:xv_train.shape[0]]

In [44]:
print(len(x_train))  # Ensure it matches len(y_train)
print(len(y_train))

21001
21001


In [45]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [46]:
pred_lr=LR.predict(xv_test)

In [47]:
LR.score(xv_test, y_test)

0.8403085273532352

In [50]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      5852
           1       1.00      0.03      0.05      1149

    accuracy                           0.84      7001
   macro avg       0.92      0.51      0.48      7001
weighted avg       0.87      0.84      0.77      7001



In [51]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [52]:
pred_dt = DT.predict(xv_test)

In [53]:
DT.score(xv_test, y_test)

0.8083130981288388

In [54]:

print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.87      0.90      0.89      5852
           1       0.40      0.34      0.37      1149

    accuracy                           0.81      7001
   macro avg       0.64      0.62      0.63      7001
weighted avg       0.80      0.81      0.80      7001



In [55]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction:{}".format(output_lable(pred_LR[0]), output_lable(pred_DT[0]), )
                                                                                                      )

In [64]:
news = str(input())
manual_testing(news)

Republicans Are Freaking Out About The ‘Dysfunction’ In The White House (VIDEO),"Sen.


LR Prediction: Fake News 
DT Prediction:Fake News


In [69]:
news = str(input())
manual_testing(news)

Exclusive: U.S. memo weakens guidelines for protecting immigrant children in court,"NEW YORK (Reuters) - The U.S. Justice Department has issued new guidelines for immigration judges that remove some instructions for how to protect unaccompanied juveniles appearing in their courtrooms. A Dec. 20 memo, issued by the Executive Office for Immigration Review (EOIR) replaces 2007 guidelines, spelling out policies and procedures judges should follow in dealing with children who crossed the border illegally alone and face possible deportation.  The new memo removes suggestions contained in the 2007 memo for how to conduct “child-sensitive questioning” and adds reminders to judges to maintain “impartiality” even though “juvenile cases may present sympathetic allegations.” The new document also changes the word “child” to “unmarried individual under the age of 18” in many instances. (Link to comparison: tmsnrt.rs/2BlT0VK May 2007 document: tmsnrt.rs/2BBR8wj December 2017 document: tmsnrt.rs/2C2s